In [1]:
import netCDF4
%pylab inline
import pyresample as pr
from numpy import array
import numpy as np
import os

Populating the interactive namespace from numpy and matplotlib


In [15]:
def get_reprojected_array(infile):
    dataset = netCDF4.Dataset(infile)

    lat_name = 'latitude'
    lon_name = 'longitude'
    lat = dataset.variables[lat_name][:]
    lon = dataset.variables[lon_name][:]

    pp_name = 'icecon'
    fill_value = dataset.variables[pp_name]._FillValue

    scale_factor = dataset.variables[pp_name].scale_factor

    #read parameter data and remove singleton dimensions:
    data_array = dataset.variables[pp_name][:].squeeze()

#     data_array = ma.masked_where(logical_or(data_array == fill_value, data_array is None), data_array)
#     data_array = ma.masked_where(logical_or(data_array == 0, data_array is None), data_array)

    data = data_array# * scale_factor

    proj = {'name': 'epsg_4326'}#, 'inch_size': (36.00, 36.00)}

    grid_def = pr.geometry.GridDefinition(lons=lon, lats=lat)

    areas_path = '/media/SOLabNFS/http/public/daniel/areas.cfg'
    area_def = pr.utils.load_area(areas_path, '%s_crude' % (proj['name']))

    area_def.x_size = 3600
    area_def.y_size = 3600
    area_def.shape = (area_def.y_size, area_def.x_size)
    area_def.size = area_def.y_size * area_def.x_size

    area_def.pixel_size_x = (area_def.area_extent[2] - area_def.area_extent[0]) / float(area_def.x_size)
    area_def.pixel_size_y = (area_def.area_extent[3] - area_def.area_extent[1]) / float(area_def.y_size)

    result = pr.kd_tree.resample_nearest(grid_def, data, area_def,
                                         radius_of_influence=80000,
                                         fill_value=int(-1), nprocs=4)
    dataset.close()
    return result

In [3]:
def write_array_to_nc(data_array, nc_path):    
    lats = map(lambda x: x/20.0, range(-1800, 1800))
    lats = array(lats)
    lats += 1/40.0
    
    lons = map(lambda x: x/10.0, range(-1800, 1800))
    lons = array(lons)
    lons += 1/20.0

    dataset = netCDF4.Dataset(nc_path, 'w', format='NETCDF4')
    dataset.createDimension('lat', len(lats))
    dataset.createDimension('lon', len(lons))

    ice_var = dataset.createVariable('icecon', 'f4', ('lon','lat'),
                                     zlib=True, complevel=6)
    ice_var[:] = data_array[:]
    lat_var = dataset.createVariable('lat', 'f4', ('lat',),
                                     zlib=True, complevel=6)
    lat_var[:] = lats[:]
    lon_var = dataset.createVariable('lon', 'f4', ('lon',),
                                     zlib=True, complevel=6)
    lon_var[:] = lons[:]
    dataset.close()


In [18]:
src_dir = '/media/SOLabNFS/store/satellite/asi_amsre_iceconc/'
out_dir = '/media/SOLabNFS2/tmp/asi_amsre_iceconc_ik/'
if not os.path.isdir(out_dir):
    os.makedirs(out_dir)

for root, dirs, files in os.walk(src_dir):
    print 'Start %s' % root
    base_name = os.path.basename(root)
    if base_name.isdigit():
        base_name = int(base_name)
    else:
        continue
    if int(base_name) == 2007 or int(base_name) == 2011:
        for filename in files:
            if not filename.startswith('asi-n'):
                continue
            month = int(filename[14:16])
            if month < 6 or month > 10:
                continue
            print 'granule: %s' % filename
            out_nc = os.path.join(out_dir, filename)
            if os.path.isfile(out_nc):
                continue
            reprojected_array = get_reprojected_array(os.path.join(root, filename))
            write_array_to_nc(reprojected_array, out_nc)

Start /media/SOLabNFS/store/satellite/asi_amsre_iceconc/
Start /media/SOLabNFS/store/satellite/asi_amsre_iceconc/2008
Start /media/SOLabNFS/store/satellite/asi_amsre_iceconc/2006
Start /media/SOLabNFS/store/satellite/asi_amsre_iceconc/2011
granule: asi-n6250-20110610-v5i.nc
granule: asi-n6250-20110723-v5i.nc
granule: asi-n6250-20110728-v5i.nc
granule: asi-n6250-20110921-v5i.nc
granule: asi-n6250-20110712-v5i.nc
granule: asi-n6250-20110720-v5i.nc
granule: asi-n6250-20110621-v5i.nc
granule: asi-n6250-20110908-v5i.nc
granule: asi-n6250-20110808-v5i.nc
granule: asi-n6250-20110811-v5i.nc
granule: asi-n6250-20110612-v5i.nc
granule: asi-n6250-20110803-v5i.nc
granule: asi-n6250-20110601-v5i.nc
granule: asi-n6250-20110629-v5i.nc
granule: asi-n6250-20110730-v5i.nc
granule: asi-n6250-20110605-v5i.nc
granule: asi-n6250-20110613-v5i.nc
granule: asi-n6250-20110807-v5i.nc
granule: asi-n6250-20110619-v5i.nc
granule: asi-n6250-20110831-v5i.nc
granule: asi-n6250-20110615-v5i.nc
granule: asi-n6250-201107

In [12]:
!ls -la /media/SOLabNFS/store/satellite/asi_amsre_iceconc/

ls: cannot access /media/SOLabNFS2/store/satellite/asi_amsre_iceconc/: No such file or directory
